# Data preparation 
---
Avant importation, les bases de données ont été nettoyées de la manière suivante : 
- Suppression des colonnes inutiles dans le jeu athlete_events.csv

## Médailles, athlètes, pays, années
---

In [11]:
# importation du fichier et des packages 
import pandas as pd 
pd.set_option('display.float_format', lambda x: '%.3f' % x)
df = pd.read_csv("../../Datasets_raw/olympic_games_1896_2016/athlete_events.csv", sep = ";", on_bad_lines="warn")
# on ne garde que les JO d'été et on supprime la colonne saison ensuite
df = df.loc[df["Season"].str.contains("Summer")]
df = df.drop(columns = "Season")
# virer les médailles qui sont en doublon à cause des évènements collectifs
df = df.drop_duplicates(subset=["Event", "Medal", "Year"])
# on groupe par année, médailles, et pays  
athlete_series = df.groupby(["NOC", "Year", "Medal"]).size()
df = athlete_series.to_frame(name = "Number of Medals").reset_index()
gold = df.loc[df['Year'] == 2016]
gold = gold.loc[df['NOC'] == "FRA"]
gold = gold.loc[df['Medal'] == "Gold"]
print(gold)
print(len(gold))
print(len(df))
# Pivot the dataframe to have separate columns for each medal type
df_pivot = df.pivot(index=["NOC", "Year"], columns="Medal", values="Number of Medals").reset_index().fillna(0)
df_pivot["Total"] = df_pivot["Bronze"] + df_pivot["Gold"] + df_pivot["Silver"]
#display(df_pivot)
# df_pivot.to_csv("./datasets_clean/olympics_main/event_medals.csv")

     NOC  Year Medal  Number of Medals
930  FRA  2016  Gold                10
1
2748


In [ ]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
medals = pd.read_csv("../../Datasets_raw/olympic_games_1896_2016/athlete_events.csv", sep = ";", on_bad_lines="warn")
# on ne garde que les JO d'été et on supprime la colonne saison ensuite
medals = medals.loc[medals["Season"].str.contains("Summer")]
medals = medals.drop(columns = "Season")
# virer les médailles qui sont en doublon à cause des évènements collectifs
medals = medals.drop_duplicates(subset=["Event", "Medal", "Year"])
# on groupe par année, médailles, et pays  
athlete_series = medals.groupby(["NOC", "Year", "Medal"]).size()
medals2 = athlete_series.to_frame(name = "Number of Medals").reset_index()
medals.to_csv("../David/datasets_clean/olympics_main/medals.csv")
medals2.to_csv("../David/datasets_clean/olympics_main/medals_count.csv")


## PIB 

In [35]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# pivoter table PIB 
gdp_cap = pd.read_csv("../../Datasets_raw/indicators/GDP/GDP_raw.csv", on_bad_lines= "warn" )
gdp_cap = gdp_cap.drop(columns = ["Indicator Code", "Unnamed: 67"])
gdp_pivot = gdp_cap.melt(id_vars=["Country Name", "Country Code"], value_vars=gdp_cap.columns[3:-1], var_name="Year", value_name="GDP")
gdp_pivot = gdp_pivot.dropna()
gdp_pivot['GDP/capita'] = gdp_pivot['GDP'].astype(float)/1000000
# gdp_pivot.to_csv("../../dataprep/David/datasets_clean/gdp_country.csv")

display(gdp_pivot)

Skipping line 25: expected 68 fields, saw 69
Skipping line 45: expected 68 fields, saw 69
Skipping line 46: expected 68 fields, saw 69
Skipping line 69: expected 68 fields, saw 69
Skipping line 81: expected 68 fields, saw 69
Skipping line 88: expected 68 fields, saw 69
Skipping line 98: expected 68 fields, saw 69
Skipping line 114: expected 68 fields, saw 69
Skipping line 128: expected 68 fields, saw 69
Skipping line 148: expected 68 fields, saw 69
Skipping line 195: expected 68 fields, saw 69
Skipping line 256: expected 68 fields, saw 69
Skipping line 264: expected 68 fields, saw 69



,Country Name,Country Code,Year,GDP,GDP/capita,NOC
1,Africa Eastern and Southern,AFE,1960,153696395951.599,153696.396,AFE
3,Africa Western and Central,AFW,1960,105675520010.086,105675.520,AFW
9,Argentina,ARG,1960,150797810295.884,150797.810,ARG
13,Australia,AUS,1960,205000837562.694,205000.838,AUS
14,Austria,AUT,1960,84930957865.742,84930.958,AUT
...,...,...,...,...,...,...
15681,Samoa,WSM,2021,843469448.212,843.469,WSM
15682,Kosovo,XKX,2021,7912073583.227,7912.074,XKX
15683,South Africa,ZAF,2021,353258433207.809,353258.433,ZAF
15684,Zambia,ZMB,2021,24495920766.027,24495.921,ZMB


## IDH et autres index calculés 

In [ ]:
# filtrer table IDH pour pas faire peur à Tableau et aussi la pivoter
hdi = pd.read_csv("../../Datasets_raw/indicators/HDI/HDR21-22_Composite_indices_complete_time_series.csv")
hdi = hdi.melt(id_vars=["iso3", "country"], value_vars=hdi.columns[4:-1], var_name="indicator_year", value_name="indicator_value")
hdi = hdi.loc[hdi["indicator_year"].str.contains("hdi|gdi|ihdi|gii|phdi")]
hdi = hdi[hdi["indicator_year"].str.contains("rank|diff|group|_f|_m") == False]
display(hdi)

# hdi.to_csv("../David/datasets_clean/hdi_clean.csv")


## Population

In [30]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# pivoter population 
pop_country = pd.read_csv("../../Datasets_raw/indicators/demography/pop_year.csv", on_bad_lines="warn")
pop_country = pop_country.drop(columns = ["Indicator Code", "Indicator Name", "Unnamed: 67"])
pop_country = pop_country.melt(id_vars=["Country Name", "Country Code"], value_vars=pop_country.columns[2:-1], var_name="Year", value_name="population (millions)")
pop_country = pop_country.dropna()
pop_country['population (millions)'] = pop_country['population (millions)'].astype(float)/1000000
pop_country = pop_country.drop(columns = "Country Code")
pop_country["NOC"] = pop_country["Country Name"]
pop_country["Year"] = pop_country["Year"].astype(int)
# pop_country.to_csv("../David/datasets_clean/country_poop_clean.csv")

Skipping line 25: expected 69 fields, saw 70
Skipping line 45: expected 69 fields, saw 70
Skipping line 46: expected 69 fields, saw 70
Skipping line 69: expected 69 fields, saw 70
Skipping line 81: expected 69 fields, saw 70
Skipping line 88: expected 69 fields, saw 70
Skipping line 98: expected 69 fields, saw 70
Skipping line 114: expected 69 fields, saw 70
Skipping line 128: expected 69 fields, saw 70
Skipping line 148: expected 69 fields, saw 70
Skipping line 195: expected 69 fields, saw 70
Skipping line 256: expected 69 fields, saw 70
Skipping line 264: expected 69 fields, saw 70



## Préparation des données pour modèle de régression linéaire 
---

In [42]:
# dans un premier temps on fusionne les valeurs de PIB / pays / année et celles des médailles
df_pivot["Year"] = df_pivot["Year"].astype(int)
gdp_pivot["Year"] = gdp_pivot["Year"].astype(int)
gdp_pivot["NOC"] = gdp_pivot["Country Code"]
df_reg = pd.merge(df_pivot, gdp_pivot, on = ["Year", "NOC"], how = "left")
# supprimer les valeurs nulles 
df_reg = df_reg.dropna(subset=["GDP"])
# on ajoute la population 
df_reg = pd.merge(df_reg, pop_country, on = ["Year", "NOC"], how = "left")
df_reg = df_reg.drop(columns = ["Country Name_y", "Country Code"])
df_reg
# on ajoute le PIB/hab 
df_reg["GDP/capita"] = df_reg["GDP"] / (df_reg["population (millions)"] * 1000000)
display(df_reg)
# df_reg.to_csv("./datasets_clean/linear_reg/linear_reg.csv")



,NOC,Year,Bronze,Gold,Silver,Total,Country Name_x,GDP,GDP/capita,population (millions)
0,ARG,1960,1.000,0.000,1.000,2.000,Argentina,150797810295.884,7410.305,20.350
1,ARG,1964,0.000,0.000,1.000,1.000,Argentina,164381681831.918,7572.231,21.708
2,ARG,1968,2.000,0.000,0.000,2.000,Argentina,195305461613.156,8450.037,23.113
3,ARG,1972,0.000,0.000,1.000,1.000,Argentina,237021461356.448,9630.010,24.613
4,ARG,1988,1.000,0.000,1.000,2.000,Argentina,293550663101.649,9262.964,31.691
...,...,...,...,...,...,...,...,...,...,...
508,UZB,2000,1.000,1.000,1.000,3.000,Uzbekistan,31472652457.938,1276.760,24.650
509,UZB,2004,0.000,2.000,1.000,3.000,Uzbekistan,38174996010.771,1475.970,25.864
510,UZB,2008,2.000,1.000,2.000,5.000,Uzbekistan,52362657828.629,1917.849,27.303
511,UZB,2012,1.000,1.000,0.000,2.000,Uzbekistan,70106848793.850,2354.594,29.774
